In [1]:
import pickle
from engines import databasefunctions
from datetime import datetime

In [2]:
class GetInformation:
    def __init__(self, api=None):
        if api == None:
            with open('dadosemacao_logged', 'rb') as filee:
                api = pickle.load(filee)
        
        self.api = api
        self.database = databasefunctions.Database()
        self.username = self.database.getProfile()[0]
        
    def __call__(self):
        if self.username != None:
            infos = {}
            infos['profile_info'] =  self.getUserInfo()
            
            assert infos['profile_info'] != None, 'Algum problema com o getUserInfo'
            self.database.updateProfile(self.username)
            
            infos['feed'] = self.getTotalFeed(infos['profile_info']['pk'])
            feed_cleaned = [self.getMediaInformation(media) for media in infos['feed']]

            self.database.saveFeed(feed_cleaned)
        else:
            print('nao ha usuario para pegar informacoes')

    def getUserInfo(self):
        if self.api.searchUsername(self.username):
            instagramProfileInfo = self.api.LastJson['user']

            #chaves que iremos pegar do json que o instagram devolve com informacoes do usuario 
            self.profile_info = {'username': '', 'address_street': '', 'biography': '', 'category': '', 'city_id': '', 'city_name': '', 'contact_phone_number': '',
                           'external_url': '', 'follower_count': '', 'following_count': '', 'full_name': '', 'profile_pic_url': '', 'zip': '', 
                           'is_business': '', 'is_private': '', 'media_count': '', 'pk': '', 'public_email': '', 'total_igtv_videos': '', 'usertags_count': ''}

            for key in self.profile_info.keys():
                 self.profile_info[key] = instagramProfileInfo[key] if key in instagramProfileInfo.keys() else None 

            self.database.saveProfileInfo(self.profile_info) #salva as informacoes no banco
            print('profile info saved in database')
            return self.profile_info

    def getMediaInformation(self, media):
        self.media_info = {'text': '', 'pk': '', 'usertags': '', 'usertags_id': '', 'status': '', 'code': '', 'comment_count': '', 'url': '', 'like_count': '', 'taken_at': '',
             'username': '', 'location': '', 'location_id': '', 'location_name': ''}

        for key in self.media_info.keys():
            try:
                if key == 'location' and key in media.keys():
                    self.media_info[key] = str(media[key])

                elif key == 'usertags' and 'usertags' in media.keys():
                    self.media_info[key] = len(media[key]['in'])

                elif key == 'comment_count' and key in media.keys() and media['comment_count'] > 0:
                    self.media_info[key] = media[key]

                elif key == 'like_count' and key in media.keys() and media['like_count'] > 0:
                    self.media_info[key] = media[key]

                elif key == 'taken_at':
                    self.media_info[key] = datetime.fromtimestamp(media[key]).strftime('%y/%m/%d %H:%M:%S')

                elif key == 'text' and key != None and media['caption'] !=None :
                    self.media_info[key] = media['caption'][key]        

                elif key == 'username':
                    self.media_info[key] = media['user'][key]
                elif key == 'url':
                    self.media_info[key] = media['image_versions2']['candidates'][0][key]
                elif key in media.keys():
                    self.media_info[key] = media[key]
            except KeyError:
                pass
            except BaseException as error:
                #self.database.saveError(error, 'getting media info')
                print('Error ocurred in: getMediaInformation --> {}'.format(error))
        return self.media_info
    
    def getTotalFeed(self, user_id):
        feed = []
        next_max_id = True
        while next_max_id:
            if next_max_id is True:
                next_max_id = ''

            _ = self.api.getUserFeed(user_id, maxid=next_max_id)
            feed.extend(self.api.LastJson['items'])
            next_max_id = self.api.LastJson.get('next_max_id', '')

        return feed


In [3]:
if __name__ == '__main__':
    gt = GetInformation()
    gt()

Fail to import moviepy. Need only for Video upload.
profile info saved in database
